# Nestor Catano (nestor.catano@berkeley.edu)
## Prototype implementation of the Chaum–Pedersen Protocol
## Based on the book: Cryptography: An Introduction, 3rd edition, by Nigel Smart

Chaum-Pedersen protocol is a zero-knowledge protocol (ZKP). The protocol considers two actors that interact, a prover (Peggy) and a verifier (Kevin). Peggy wants to convince Kevin that she knows something. She is called a prover because she needs to conduct and enclose a correctness proof. He is called a verifier because needs to check her proof. The protocol is run over four steps. A **commitment-step** in which she identifies herself. A **challenge-step** in which he generates a challenge for her to solve. And, a **response-step** in which she solves the challenge and sends him a proof of it. Finally, he needs to **verify the challenge** and act accordingly. Refer to the book above for a detailed explanation of each of these steps (Chapter 25, Section 3.2).

Following page 375 of the said book, a three round protocol has the **special soundness** property if given two protocol runs with the same commitments <code>r</code>, but different challenges <code>c</code> one can recover the prover's secrete <code>x</code>. Suppose that we have two protocol runs <code>(r,c,s)</code> and <code>(r,c',s')</code>.

This NoteBook contains an implementation of Chaum-Pedersen protocol. We have manually written unit tests that increase our confidence in the correctness of our implementation. We further discuss underlying challenges of the implementation. 



## Architecture of the Chaum-Pederson protocol

Our implementation has 3 parts that run independently, namely, the **server side** (Kevin, the verifier), the **client side** (Peggy, the prover), and some  **common-info** that's shared between the server and the client. These three parts can be implemented, for instance, using a distributed architecture, or in the cloud. Our implementation is a skeleton prototype implementation that sheers light on the challenges and issues that a full-fledged implementation would pose. This is a prototype implementation, so we do not necessarily claim that Python itself is the best choice for a target language implementation. Our implementation is an object-oriented implementation. We do not claim object orientation is the best-suited programming paradigm for implementing Chaum-Pederson; in fact, many Python programs are written using a functional style of programming. Nevertheless, object orientation is well-suited for implementing state interaction, e.g., if one wants to make sure that a **response-step** is only executed right after a **challenge-step**.

We use Python's **numpy** library (<code>np</code>) to conduct mathematical calculations over integer number, and to generate prime numbers. The code we use for generating large prime numbers is not ours. We use <code>np.int64</code> to represent integer numbers.

Let's first show some information on how **numpy** represents integers of size 64, which we will be using to represent public and private information of our ZKP (zero-knowledge-protocol).

In [1]:
import numpy as np

In [2]:
ii64 =  np.iinfo(np.int64)
print(ii64)

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------



## Prime numbers
We need to implement code that generates large number primes. The code below is not ours, it's been taken from [here](https://github.com/NikolaiT/Large-Primes-for-RSA/blob/master/generate_primes.py). It generates a list of prime numbers of some size. This code should be run once. 

In [3]:
import re
import random
import math

"""
Generate prime numbers with the Miller-Rabin Primality Test.
For example useful for RSA prime number generation.
    sys	0m0.020s
"""

def fermat_primality_test(p, s=5):
    """
    a^(p-1) ≡ 1 mod p
    Input: prime candidate p and security paramter s
    Output: either p is a composite (always trues), or
            p is a prime (with probability)
    """
    if p == 2:
        return True
    if not p & 1: # if p is even, number cant be a prime
        return False

    for i in range(s):
        a = random.randrange(2, p-2)
        x = pow(a, p-1, p) # a**(p-1) % p
        if x != 1:
            return False
    return True

def square_and_multiply(x, k, p=None):
    """
    Square and Multiply Algorithm
    Parameters: positive integer x and integer exponent k,
                optional modulus p
    Returns: x**k or x**k mod p when p is given
    """
    b = bin(k).lstrip('0b')
    r = 1
    for i in b:
        r = r**2
        if i == '1':
            r = r * x
        if p:
            r %= p
    return r

def miller_rabin_primality_test(p, s=5):
    if p == 2: # 2 is the only prime that is even
        return True
    if not (p & 1): # n is a even number and can't be prime
        return False

    p1 = p - 1
    u = 0
    r = p1  # p-1 = 2**u * r

    while r % 2 == 0:
        r >>= 1
        u += 1

    # at this stage p-1 = 2**u * r  holds
    assert p-1 == 2**u * r

    def witness(a):
        """
        Returns: True, if there is a witness that p is not prime.
                False, when p might be prime
        """
        z = square_and_multiply(a, r, p)
        if z == 1:
            return False

        for i in range(u):
            z = square_and_multiply(a, 2**i * r, p)
            if z == p1:
                return False
        return True

    for j in range(s):
        a = random.randrange(2, p-2)
        if witness(a):
            return False

    return True

def generate_primes(n=512, k=1):
    """
    Generates prime numbers with bitlength n.
    Stops after the generation of k prime numbers.
    Caution: The numbers tested for primality start at
    a random place, but the tests are drawn with the integers
    following from the random start.
    """
    assert k > 0
    assert n > 0 and n < 4096

    # follows from the prime number theorem
    necessary_steps = math.floor( math.log(2**n) / 2 )
    # get n random bits as our first number to test for primality
    x = random.getrandbits(n)

    primes = []

    while k>0:
        if miller_rabin_primality_test(x, s=7):
            primes.append(x)
            k = k-1
        x = x+1

    return primes

## Generating prime number <code>p</code>
In the Chaum-Pedersem protocol, calculations are conducted modulo <code>p</code> (a prime number). We chose <code>bitlength = 32</code> to represent our integers. Object <code>p</code> is part of the <code>CommonInfo</code> object (discussed larter on), which is available to both **the verifier** and **the prover**. 

In [4]:
bitlength = 16
numprimes = 1
primes = generate_primes(n=bitlength,k=numprimes)
p = primes[0]
q = np.int64(p) # converting the prime number from int to int64
print(type(q))
q

<class 'numpy.int64'>


19333

## The Common-Info
Class <code>CommonInfo</code> implements functionality that stores and manipulates common information shared between **the server** and **the client**. The constructor of the class (see <code>__init__(self,m,q)</code>) is parametrised by a maximum integer size value <code>m</code> and a prime number <code>q</code>. Computations are conducted modulo <code>q</code>. Hence, one needs to create (a list of) prime number before instantiating this class. Class <code>CommonInfo</code> gathers information for public functions <code>g</code> and <code>h</code> (used for ciphering secrete information).

The class implements a <code>exp_mod_q(a,b)</code> function that calculates <code>a^b mod q</code>. This function implements the **repeated squaring** algorithm. We have optimised the code of this function to prevent integer overflow exceptions modulo <code>q</code>. The optimisation is documented within the code.

In [5]:
class CommonInfo:
  """ This class encloses the public info that's available to both the prover  
  and the verifier.
  """
  def __init__(self,m,q):
    """ m is the maximum size of _g and _h.
        g is a padding function, a number.
        h is a hashing function, a number.
        q is a prime number. Multiplications and additions are conducted 
        modulo _q.
    """
    self._g = np.random.randint(0, m, dtype=np.int64)
    self._h = np.random.randint(0, m, dtype=np.int64)
    self._m = m # maximum integer number
    self._q = q # operations are modulo p

  def get_g(self):
    """ It returns function g.
    """
    return self._g

  def get_h(self):
    """ It returns function h.
    """
    return self._h

  def get_q(self):
    """ It returns prime number q.
    """
    return self._q

  def get_maxsize(self):
    """ It returns the maximum integer size m.
    """
    return self._m

  def exp_mod_q(self,a,b):
    """It calculates a^b (mod q) using the method for
    repeated squaring."""

    q = self.get_q()
    t = 1;
    while(b > 0):
      if (b % 2 != 0): # if b is odd
        # we have modidfied two lines below so that instead of multiplying two 
        # numbers and then calculating the modulo-q value, we calculate
        # the modulo-q values before and after multiplying the two numbers.
        
        #t = (t * a) % q;
        t = ((t % q) * (a % q)) % q

      #a = (a * a) % q;
      a = ((a%q) * (a%q)) % q
      b = int(b / 2);
    return t % q


# The Prover
Class <code>Peggy</code> implements **the prover**. The class implements two steps of the Chaum-Pedersem protocol, namely the **commitment-step** through the function <code>commitment(self)</code>, and the **response-step** through the function <code>solve_challenge(self,c)</code>. We optimised the code of the <code>solve_challenge</code> function (details are explained within the code).

In [6]:
class Peggy: 
  """ This class implements the prover. The class has 3 fields, x, k, and pinfo, 
  which are private to the class; pinfo is the public info shared between Peggy
  and the server. Variable x is peggy's password, which is secret and created 
  randomly. Variabke k is used to conceal peggy's secret password. Peggy's
  password can be reset.

  This class implements two steps of the Chaum-Pedersem protocol, namely, the
  commitment-step and the response-step.
  """
  def __init__(self,desc,pinfo):
    """x is Peggy's password, initally created as a random value.
    k is a padding variable used to conceal x.
    pinfo is an object reference to the public information that Peggy shares
    with the verifier.
    desc is a string prover representation
    """
    m = pinfo.get_maxsize()
    msqrt = np.int64(np.sqrt(np.sqrt(m)))
    self._x = np.random.randint(0, msqrt, dtype=np.int64) 
    self._k = np.random.randint(msqrt, m, dtype=np.int64) 
    self._pinfo = pinfo
    self._desc = desc

  def __str__(self):
    """String representation of the class."""
    return self._desc 
  
  def reset_k(self, k):
    """It resets k's value."""
    self._k = np.random.randint(0, self._pinfo.get_maxsize(), dtype=np.int64)
  
  def set_x(self, x):
    """It resets x."""
    self._x = x

  def commitment(self,verbose=False):
    """prover's commitment step."""
    g = self._pinfo.get_g() # g is public
    h = self._pinfo.get_h() # h is public
    k = self._k # k is private to peggy
    x = self._x # x is private to peggy

    # r1,r2,y1,y2 are transmitted to the server
    r1 = self._pinfo.exp_mod_q(g,k)
    r2 = self._pinfo.exp_mod_q(h,k)
    y1 = self._pinfo.exp_mod_q(g,x)
    y2 = self._pinfo.exp_mod_q(h,x)
    
    # cmt is a dictionary (a hash map)
    cmt = {'r1':r1,'r2':r2,'y1':y1,'y2':y2}

    if verbose == True:
      print('x: ', x)
      print('k: ', k)
      print('r1: ', r1)
      print('r2: ', r2)
      print('y1: ', y1)
      print('y2: ', y2)

    return cmt
  
  def solve_challenge(self,c):
    """prover's response step."""
    k = self._k
    x = self._x 
    q = self._pinfo.get_q()
    c_x = c*x
    # this step is tuned elsewhere, notice that we are using integer 
    # arithmetic here, hence s cannot be negative, otherwise, g^s and h^s will 
    # mistakenly be approximated to 1. We made sure elsewhere that 
    # the values of c and x are such that their multiplication is less than k.
    s = k - c_x 
    return s

# The Server
Code that implements the functionality of **the server**. Function <code>protocol(self,prover)</code> implements the Chaum-Pedersem protocol. Function <code>generate_challenge(self,prover)</code> implements the **challenge_step**, and function <code>check_response(self,prover,cmt,s,c)</code> is the last step of the protocol whereby the prover checks if the verifier solved the challenge adequately.

In [7]:
class Kevin:
  """ This class implements the server. """
 
  def __init__(self,pinfo):
    """
    pinfo is an object reference to the common information shared with 
    clients (provers).
    c is a generated challenge
    """
    self._pinfo = pinfo
    self._c = None

  def __str__(self):
    """String representation of the class."""
    return "Verifier:"

  def generate_challenge(self,prover):
    """prover is the client for which the challenge is generated.
    It returns a random challenge. 
    """
    m = pinfo.get_maxsize()
    msqrt = np.int64(np.sqrt(np.sqrt(m)))
    c = np.random.randint(0, msqrt, dtype=np.int64) 
    self._c = c
    return c

  def check_response(self,prover,cmt,s,verbose=False):
    """prover is the client for whom the response is checked.
    cmt is the prover's response generated during the commitment step.
    """
    g = self._pinfo.get_g()
    h = self._pinfo.get_h()
    q = self._pinfo.get_q()
    c = self._c # challenge generated for prover
    #
    r1 = cmt['r1']
    r2 = cmt['r2']
    y1 = cmt['y1']
    y2 = cmt['y2']
    #
    g_s =  self._pinfo.exp_mod_q(g,s)
    h_s =  self._pinfo.exp_mod_q(h,s)
    y1_c =  self._pinfo.exp_mod_q(y1,c)
    y2_c =  self._pinfo.exp_mod_q(y2,c)
    #
    r1_c = (g_s*y1_c)%q
    r2_c = (h_s*y2_c)%q

    if verbose==True:
      print('s: ',s)
      print('c: ',c)
      print('g_s: ',g_s)
      print('h_s: ',h_s)
      print('y1_c: ',y1_c)
      print('y2_c: ',y2_c)

    return r1==r1_c and r2==r2_c

  def protocol(self,prover,verbose=False):
    # commitment step
    cmt = prover.commitment(verbose)
    # challenge step
    c = self.generate_challenge(prover)
    # response step
    s = prover.solve_challenge(c)
    # verification step
    resp = self.check_response(prover,cmt,s,verbose)
    return resp



## Simple prototocol interaction (1)

In [8]:
# the common info
maxsize = 2**bitlength
pinfo = CommonInfo(maxsize,q)

# the prover 
peggy = Peggy('peggy',pinfo)

# the verifier
kevin = Kevin(pinfo) 
resp = kevin.protocol(peggy,verbose=True)

print('g: ' ,pinfo.get_g())
print('h: ' ,pinfo.get_h())
print('q: ' ,pinfo.get_q())

print('protocol\'s response: ', resp)

x:  5
k:  46390
r1:  18891
r2:  12494
y1:  14719
y2:  7417
s:  46320
c:  14
g_s:  5212
h_s:  13699
y1_c:  2500
y2_c:  12358
g:  3307
h:  11925
q:  19333
protocol's response:  True


# Unit testing (part one)
The single test above can be reproduced using the <code>unittest</code> framework of Python.

In [9]:
import unittest

class TestStringMethods(unittest.TestCase):

    def test_simple(self):
      peggy = Peggy('peggy',pinfo)
      kevin = Kevin(pinfo) 
      resp = kevin.protocol(peggy)
      self.assertTrue(resp)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


# Discussion (part one)

We next discuss some issues of our implementation, suggest some solutions, and offer perspectives on related challenges. 

---
*   The server should be parametrised by the registered provers (clients). Hence, every time a prover calls a method of the verifier, the call and every data generated by the verifier should be registered and tracked back by the particular prover instance.

* One particular issue in our implementation is that the verifier is stateless, which means that, for instance, the verifier can perform a challenge-step without a prior prover's commitment-step. More concretely, a challenge should be issued to a prover only if the prover had performed a commitment-step during the previous state. Otherwise, we must allow a prover state-transition interleaving semantics for different verifiers.

* Provers should be ascribed to one or several verifiers.
---

Let's summarise the changes made to the prover:

*   Variable <code>self._challenges</code> is a set of pairs that records challenge per prover instance.
*   Function <code>generate_challenge(self,prover)</code> associates <code>prover</code> with the generated challenge.
* Constructor <code>__init__(self,pinfo,provers)</code> is now parametrised by a list of <code>provers</code>.
* <code>check_response(self,prover,cmt,s)</code> includes code <code>if c == None: return False</code> that returns <code>False</code> when no challenge associated to <code>prover</code> exists. This is the case for instance, when some prover makes a commitment-step but a second prover attempts to performm a solve-challenge step (see <code>test_03</code> below).
* <code>protocol(self,prover)</code> includes code <code>if not( prover in self._provers): return False</code> that returns <code>False</code> if <code>prover</code> is not a registered prover.


In [10]:
class Kevin:
  """ This class implements the server. """
 
  def __init__(self,pinfo,provers):
    """
    pinfo is an object reference to the common information shared with 
    clients (provers).
    """
    self._pinfo = pinfo
    # set of provers registered with the sever
    self._provers = provers
    # challenges is a partial function; we model it as a 
    # set of pairs (prover,challenge) 
    self._challenges = set([])

  def __str__(self):
    """String representation of the class."""
    return "Verifier:"

  def generate_challenge(self,prover):
    """prover is the client for which the challenge is generated.
    The challenge c generated is stored as self.challenges(prover) = c
    """
    m = pinfo.get_maxsize()
    msqrt = np.int64(np.sqrt(np.sqrt(m)))
    c = np.random.randint(0, msqrt, dtype=np.int64) 
    self._challenges.add((prover,c))
    return c

  def get_challenge(self,prover):
    """it returns the challenge associated with the parameter prover (if any).
    """
    for (p,c) in self._challenges:
      if p == prover: return c

  def set_challenge(self,prover,c):
    """it sets c as the challenge for the prover.
    """
    # to remove any previous association 
    challenges = self._challenges.copy()
    for (p1,c1) in challenges:
      if p1 == prover: 
        self._challenges.remove((p1,c1))

    # to add the challenge
    self._challenges.add((prover,c))

  def check_response(self,prover,cmt,s):
    """prover is the client for whom the response is checked.
    cmt is the prover's response generated during the commitment step.
    """
    g = self._pinfo.get_g()
    h = self._pinfo.get_h()
    q = self._pinfo.get_q()
    c = self.get_challenge(prover) # challenge generated for the 'prover'
    
    # new check: 
    # we check that a challenge c for prover exists
    if c == None: 
      return False
    #
    r1 = cmt['r1']
    r2 = cmt['r2']
    y1 = cmt['y1']
    y2 = cmt['y2']
    #
    g_s =  self._pinfo.exp_mod_q(g,s)
    h_s =  self._pinfo.exp_mod_q(h,s)
    y1_c =  self._pinfo.exp_mod_q(y1,c)
    y2_c =  self._pinfo.exp_mod_q(y2,c)
    #
    r1_c = (g_s*y1_c)%q
    r2_c = (h_s*y2_c)%q

    return r1==r1_c and r2==r2_c

  def protocol(self,prover):
    # new check:
    # we check the prover is a registed prover
    if not( prover in self._provers ):
      return False

    # commitment step
    cmt = prover.commitment()
    # challenge step
    self.generate_challenge(prover)
    c = self.get_challenge(prover)
    # response step
    s = prover.solve_challenge(c)
    # verification step
    resp = self.check_response(prover,cmt,s)
    return resp

## Simple protocol interaction (2)

Let's first repeat the same unit-testing experiment before diving into something more complicated with <code>unittest</code>. 

In [11]:
# the common info
maxsize = 2**bitlength
pinfo = CommonInfo(maxsize,q)

# the prover 
peggy = Peggy('peggy',pinfo)
provers = [peggy]

# the verifier
kevin = Kevin(pinfo,provers) 
resp = kevin.protocol(peggy)
print('protocol\'s response: ', resp)

protocol's response:  True


# Unit Testing (part two)

In [12]:
import unittest

class TestStringMethods(unittest.TestCase): 

    def test_01(self):
      """|
      it tests a simple interaction between prover and verifier
      """
      peggy = Peggy('peggy',pinfo)
      kevin = Kevin(pinfo,[peggy]) 
      resp = kevin.protocol(peggy)
      self.assertTrue(resp)

    def test_02(self):
      """
      it's a more granular version of test_01
      """
      peggy = Peggy('peggy',pinfo) # the prover
      kevin = Kevin(pinfo,[peggy]) # the verifier

      # commitment step
      cmt = peggy.commitment()
      # challenge step
      kevin.generate_challenge(peggy)
      c = kevin.get_challenge(peggy)
      # response step
      s = peggy.solve_challenge(c)
      # verification step
      resp = kevin.check_response(peggy,cmt,s)
      self.assertTrue(resp)

    def test_03(self):
      """
      it tests that the prover cannot solve a challenge other than the
      one generated by the verifier for the prover
      """
      peggy = Peggy('peggy',pinfo)
      kevin = Kevin(pinfo,[peggy]) 

      # commitment step
      cmt = peggy.commitment()
      # challenge step
      kevin.generate_challenge(peggy)
      c = kevin.get_challenge(peggy)
      # response step
      s = peggy.solve_challenge(c+1) # it tries to solve a different challenge
      # verification step
      resp = kevin.check_response(peggy,cmt,s)
      self.assertFalse(resp)
      self.assertFalse((c+1) == kevin.get_challenge(peggy))
    
    def test_04(self):
      """
      it tests that only peggy (and not alice) can solve a challenge
      for a commitment that peggy made.
      """
      peggy = Peggy('peggy',pinfo) # this is the first prover
      alice = Peggy('alice',pinfo) # this is a second prover
      kevin = Kevin(pinfo,[peggy,alice]) # this is the verifier

      # commitment step
      cmt = peggy.commitment()
      # challenge step
      kevin.generate_challenge(peggy)
      c = kevin.get_challenge(peggy)
      # response step
      s = peggy.solve_challenge(c)
      # verification step
      resp = kevin.check_response(alice,cmt,s) # another prover tries to solve
                                              # the challenge
      self.assertFalse(resp)

    def test_05(self):
      """
      it tests that non-registered users do not hold a chance to
      successfully go through the protocol.
      """
      peggy = Peggy('peggy',pinfo)
      kevin = Kevin(pinfo,[]) # peggy IS NOT registered with the server
      resp = kevin.protocol(peggy)
      self.assertFalse(resp) 

    def test_06(self):
      """|
      it conductes a few checks related with the generation of challenges
      """
      peggy = Peggy('peggy',pinfo)
      kevin = Kevin(pinfo,[peggy])
      self.assertTrue(kevin.get_challenge(peggy) == None) 
      c1 = 33
      kevin.set_challenge(peggy,c1)
      self.assertTrue(kevin.get_challenge(peggy) == c1)  
      c2 = 55
      kevin.set_challenge(peggy,c2)
      self.assertTrue(kevin.get_challenge(peggy) == c2) 


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


......
----------------------------------------------------------------------
Ran 6 tests in 0.030s

OK
